In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import json
# Import API key
import gmaps
# Google developer API key
from datetime import date
from config import gkey
from pprint import pprint
# Configure gmaps
gmaps.configure(api_key=gkey)


In [2]:
#data provided by emily and harlan https://github.com/NABSA/gbfs/blob/master/systems.csv
bigdata_df=pd.read_csv("big_data.csv")
bigdata_df


,Country Code,Name,Location,System ID,URL,Auto-Discovery URL
0,AE,ADCB Bikeshare,"Abu Dhabi, AE",ABU,https://www.bikeshare.ae/,https://api-core.bikeshare.ae/gbfs/gbfs.json
1,AE,Careem BIKE,"Dubai, AE",careem_bike,https://www.careem.com/en-ae/careem-bike/,https://dubai.publicbikesystem.net/ube/gbfs/v1/
2,AR,Ecobici,"Buenos Aires, AR",bike_buenosaires,https://www.buenosaires.gob.ar/ecobici,https://buenosaires.publicbikesystem.net/ube/g...
3,AR,MiBiciTuBici,"Rosario, Santa Fe, AR",biketobike,https://www.mibicitubici.gob.ar/,https://www.mibicitubici.gob.ar/opendata/gbfs....
4,AR,Movo Buenos Aires,"Buenos Aires, AR",movo_bue,https://movo.me/ar,https://gbfs.movo.me/v1/buenosaires/gbfs.json
...,...,...,...,...,...,...
530,US,Wheels Miami,"Miami, FL",9f74fa80-fb2f-48dc-b2d8-01a889ac008a,https://takewheels.com/,https://mia-gbfs.getwheelsapp.com/gbfs.json
531,US,Wheels Orlando,"Orlando, FL",9f74fa80-fb2f-48dc-b2d8-01a889ac008a,https://takewheels.com/,https://orlando-gbfs.getwheelsapp.com/gbfs.json
532,US,Wheels San Diego,"San Diego, CA",9f74fa80-fb2f-48dc-b2d8-01a889ac008a,https://takewheels.com/,https://sandiego-gbfs.getwheelsapp.com/gbfs.json
533,US,Wheels Tallahassee,"Tallahassee, FL",9f74fa80-fb2f-48dc-b2d8-01a889ac008a,https://takewheels.com/,https://tallahassee-gbfs.getwheelsapp.com/gbfs...


In [3]:
#getting data from only US cities
us_data=bigdata_df.loc[(bigdata_df["Country Code"]=="US")]
us_data

,Country Code,Name,Location,System ID,URL,Auto-Discovery URL
331,US,Austin B-cycle,"Austin, TX",bcycle_austin,http://austinbcycle.com,https://gbfs.bcycle.com/bcycle_austin/gbfs.json
332,US,Aventura BCycle,"Aventura, FL",bcycle_aventura,https://aventura.bcycle.com,https://gbfs.bcycle.com/bcycle_aventura/gbfs.json
333,US,Bay Wheels,"San Francisco Bay Area, CA",BA,https://www.baywheels.com/,https://gbfs.baywheels.com/gbfs/gbfs.json
334,US,Bike Chattanooga,"Chattanooga, TN",bike_chattanooga,http://www.bikechattanooga.com/,https://chat.publicbikesystem.net/ube/gbfs/v1/
335,US,BikeLNK,"Lincoln, NE",bcycle_bikelnk,https://bikelnk.bcycle.com,https://gbfs.bcycle.com/bcycle_bikelnk/gbfs.json
...,...,...,...,...,...,...
530,US,Wheels Miami,"Miami, FL",9f74fa80-fb2f-48dc-b2d8-01a889ac008a,https://takewheels.com/,https://mia-gbfs.getwheelsapp.com/gbfs.json
531,US,Wheels Orlando,"Orlando, FL",9f74fa80-fb2f-48dc-b2d8-01a889ac008a,https://takewheels.com/,https://orlando-gbfs.getwheelsapp.com/gbfs.json
532,US,Wheels San Diego,"San Diego, CA",9f74fa80-fb2f-48dc-b2d8-01a889ac008a,https://takewheels.com/,https://sandiego-gbfs.getwheelsapp.com/gbfs.json
533,US,Wheels Tallahassee,"Tallahassee, FL",9f74fa80-fb2f-48dc-b2d8-01a889ac008a,https://takewheels.com/,https://tallahassee-gbfs.getwheelsapp.com/gbfs...


In [4]:
#getting data for bird and lime escooter
data_scoot = us_data.loc[us_data['System ID'].str.contains("spin", case=False)]
len(data_scoot)
#data_scoot
data_scoot

,Country Code,Name,Location,System ID,URL,Auto-Discovery URL
442,US,Spin Aiken,"Aiken, SC",spin aiken,https://www.spin.pm,https://gbfs.spin.pm/api/gbfs/v1/aiken/gbfs
443,US,Spin Albuquerque,"Albuquerque, NM",spin albuquerque,https://www.spin.pm,https://gbfs.spin.pm/api/gbfs/v1/albuquerque/gbfs
444,US,Spin Alexandria,"Alexandria, VA",spin alexandria,https://www.spin.pm,https://gbfs.spin.pm/api/gbfs/v1/alexandria/gbfs
445,US,Spin Ann Arbor,"Ann Arbor, MI",spin ann_arbor,https://www.spin.pm,https://gbfs.spin.pm/api/gbfs/v1/ann_arbor/gbfs
446,US,Spin Arkadelphia,"Arkadelphia, VA",spin arkadelphia,https://www.spin.pm,https://gbfs.spin.pm/api/gbfs/v1/arkadelphia/gbfs
...,...,...,...,...,...,...
518,US,Spin Washington DC,"Washington, DC",spin washington_dc,https://www.spin.pm,https://gbfs.spin.pm/api/gbfs/v1/washington_dc...
519,US,Spin West Sacramento,"West Sacramento, CA",spin west_sacramento,https://www.spin.pm,https://gbfs.spin.pm/api/gbfs/v1/west_sacramen...
520,US,Spin White Center,"White Center, WA",spin white_center,https://www.spin.pm,https://gbfs.spin.pm/api/gbfs/v1/white_center/...
521,US,Spin Wichita,"Wichita, KS",spin wichita,https://www.spin.pm,https://gbfs.spin.pm/api/gbfs/v1/wichita/gbfs


In [7]:
# #getting latitudes and longitudes for city as well as coordinates of bird scooters in each cities
# coord_length=[]
lat_list=[]
lon_list=[]
latlon = []
all_url=data_scoot["Auto-Discovery URL"]
x = 0
y = 0
bike = 0
dict = {}
city_list = []
count_list = []
city_list_coord = []

for i, row in data_scoot.iterrows():
    
    # print(base_url)
    base_url= row['Auto-Discovery URL']
    cities = row['Location']
    print(f'running record {x} on {cities}')
        
    # get URL for location
    result = requests.get(base_url).json()
    result
    # Fixed the jsondecode issue
    #response = requests.get(base_url).json()
    # print(json.dumps(response, indent=4, sort_keys=True))


    # get URL with bike status for location
    base_url1 = result['data']['en']['feeds'][1]['url']
    base_url1

    # get file with location info
    result1 = requests.get(base_url1).json()
    result1

    bike_coord = result1['data']['bikes']
    bike_coord
    print(f'there are {len(bike_coord)} bikes in record {x}')
    city_list.append(cities)
    count_list.append(len(bike_coord))
    
    for bike in bike_coord:
        
        # Add Lat info to list
        lat_list.append([bike][0]['lat'])
        lon_list.append([bike][0]['lon'])
        city_list_coord.append(cities)
    
    
    x = x+1
    time.sleep(2)
 
    
# # # latlon.append(dict)
# # #         x = x +1
# # #         y = y+1
        

        

running record 0 on Aiken, SC
there are 0 bikes in record 0
running record 1 on Albuquerque, NM
there are 0 bikes in record 1
running record 2 on Alexandria, VA
there are 4 bikes in record 2
running record 3 on Ann Arbor, MI
there are 40 bikes in record 3
running record 4 on Arkadelphia, VA
there are 0 bikes in record 4
running record 5 on Arlington, VA
there are 0 bikes in record 5
running record 6 on Asbury Park, NY
there are 0 bikes in record 6
running record 7 on Atlanta, GA
there are 762 bikes in record 7
running record 8 on Austin, TX
there are 0 bikes in record 8
running record 9 on Baltimore, MD
there are 1014 bikes in record 9
running record 10 on Boise, ID
there are 153 bikes in record 10
running record 11 on Brookline, MA
there are 0 bikes in record 11
running record 12 on Charlotte,NC
there are 222 bikes in record 12
running record 13 on Chicago, IL
there are 0 bikes in record 13
running record 14 on Cleveland, OH
there are 365 bikes in record 14
running record 15 on Columb

In [32]:
# #get for one city
lat_list=[]
lon_list=[]
x = 0
y = 0
bike = 0
dict = {}
city_list = []
count_list = []
city_list_coord = []

base_url= 'https://gbfs.spin.pm/api/gbfs/v1/white_center/gbfs'
cities = "White Center"
        
# get URL for location
result = requests.get(base_url).json()

# get URL with bike status for location
base_url1 = result['data']['en']['feeds'][1]['url']
   
# get file with location info
result1 = requests.get(base_url1).json()
 
bike_coord = result1['data']['bikes']
print(f'there are {len(bike_coord)} bikes in {cities}')
    
for bike in bike_coord:
        
    # Add Lat info to list
    lat_list.append([bike][0]['lat'])
    lon_list.append([bike][0]['lon'])
      
    
 
        

there are 24 bikes in White Center


In [33]:
#Save csv for one city
city_bird_df = pd.DataFrame({'Lat':lat_list, 'Lon': lon_list})
city_bird_df['Operator'] = "Spin"
today = date.today()
city_bird_df['Date'] = today
city_bird_df.to_csv('White_Center_Spin_Lat_Lon.csv')

In [12]:
df2 = pd.DataFrame({'Lat':lat_list, 'Lon': lon_list, 'City': city_list_coord})
df2['Operator'] = "Spin"
today = date.today()
df2['Date'] = today
df2
df2.to_csv('Geocode_Spin_Lat_Lon.csv')

In [14]:
new_list = list(zip(lat_list,lon_list))
len(new_list)

15438

In [15]:
locations = list(zip(lat_list,lon_list))
locations

[(38.83409, -77.10892),
 (38.83315, -77.08517),
 (38.83407, -77.10894),
 (38.83256, -77.05068),
 (42.25731, -83.71387),
 (42.30978, -83.71729),
 (42.24121, -83.75605),
 (42.25063, -83.74781),
 (42.28506, -83.7296),
 (42.29028, -83.73369),
 (42.25753, -83.73603),
 (42.25699, -83.73495),
 (42.24403, -83.75689),
 (42.29476, -83.73643),
 (42.2584, -83.75029),
 (42.25646, -83.76651),
 (42.28499, -83.71533),
 (42.26909, -83.7865),
 (42.25281, -83.6811),
 (42.28686, -83.71853),
 (42.28076, -83.76113),
 (42.27384, -83.7792),
 (42.24659, -83.75267),
 (42.25137, -83.70957),
 (42.24428, -83.7451),
 (42.25138, -83.70954),
 (42.25138, -83.70951),
 (42.23994, -83.75423),
 (42.26737, -83.77719),
 (42.23843, -83.74079),
 (42.28268, -83.74937),
 (42.27638, -83.72753),
 (42.28506, -83.75633),
 (42.25685, -83.73486),
 (42.24105, -83.75562),
 (42.26483, -83.74032),
 (42.24296, -83.76476),
 (42.28933, -83.75464),
 (42.27639, -83.74512),
 (42.29832, -83.76592),
 (42.29881, -83.75532),
 (42.24107, -83.75561)

In [16]:

#generating the heat map using the city(lat and lon values) with
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))